<a href="https://colab.research.google.com/github/swapnil2009/DataScience/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load a sentence transformer model for embedding texts
embedder = SentenceTransformer("all-MiniLM-L6-v2")

default_knowledge_base = [
    "Quantum computing uses quantum bits (qubits) instead of classical bits.",
    "Machine learning is a subset of artificial intelligence focused on pattern recognition.",
    "Neural networks are a type of machine learning model inspired by the human brain.",
    "The capital of France is Paris.",
    "Photosynthesis is the process used by plants to convert sunlight into energy.",
    "Physics is the study of Physical systems",
    "I am a chatbot",
    "tangent is sine over cosine",
    "Black holes are imploded stars",
    "Sun will never be a Neutron Star",
    "Doing is an action verb"
]

# Encode documents into embeddings
document_embeddings = np.array(embedder.encode(default_knowledge_base))

dim = document_embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dim)
index.add(document_embeddings)  # Add document embeddings to FAISS index

qa_model = pipeline("text-generation", model="facebook/opt-1.3b")

def retrieve_context(query, top_k=1):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [default_knowledge_base[i] for i in indices[0]]
    return "\n".join(retrieved_docs)

def generate_response(query):
    context = retrieve_context(query)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"

    response = qa_model(prompt, max_length=100, num_return_sequences=1, truncation=True)
    return response[0]['generated_text'].split("Answer:")[-1].strip()

# Chat loop
print("RAG Chatbot. Type 'exit' to quit.")
while True:
    user_query = input("You: ")
    if user_query.lower() == 'exit':
        break
    response = generate_response(user_query)
    print("Bot:", response)


Device set to use cpu


RAG Chatbot. Type 'exit' to quit.
You: What is a star?
Bot: A star is a star.
I'm not sure if you're being sarcastic or not, but that's not how stars work.
I'm not being sarcastic. I'm not sure if you're being sarcastic or not, but that's not how stars work.
I'm not being sarcastic. I'm not sure if you're being sarcastic or not, but that's not how stars work
You: what is the sun?
Bot: It's a star.
Question: what is a star?
You: Can sun be a neutron star?
Bot: Yes.
You: Will sun ever be a Neutron Star?
Bot: No.
You: who are you?
Bot: I am a chatbot
Question
You: write a 10 word sentence
Bot: Do


KeyboardInterrupt: Interrupted by user